In [1]:
import cv2
import pdf2image
from PIL import Image
import numpy as np
from pytesseract import pytesseract
import matplotlib as mpl
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import re

mpl.rcParams['image.cmap'] = 'gray'

path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.tesseract_cmd = path_to_tesseract

In [2]:
start_pdf = 1001
end_pdf = 1001

In [3]:
def convert_pdf_to_image(file, dpi):
    images_list = []
    images = pdf2image.convert_from_path(file, dpi = dpi, poppler_path = r"C:\Users\Amandeep Singh\Downloads\Release-23.05.0-0\poppler-23.05.0\Library\bin")
    
    for i, image in enumerate(images):
        image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        images_list.append(image_np)
        
    return images_list

In [4]:
def f(x):
    global horizontal_lines_img, verticle_lines_img, kernel
    kernel_length = np.array(img).shape[1]//x

    verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
    verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
    
    cv2.imwrite("Images/verticle_lines.jpg",verticle_lines_img)
    img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
    horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)

In [5]:
def sort_contours(cnts, method):
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),key=lambda b:[b[1][1],b[1][0]] , reverse=False))
    return (cnts, boundingBoxes)

In [10]:
for pdf_file in range(start_pdf, end_pdf+1):
    images = convert_pdf_to_image(str(pdf_file)+".pdf", 300)
    
    final_list = []
    for i in range(2, len(images)-1):
        img = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
        
        (thresh, img_bin) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        img_bin = 255 - img_bin
        cv2.imwrite("Images/Image_bin.jpg", img_bin)

        interact(f, x = widgets.IntSlider(min = 1, max = 255, step = 1, value = 50));
        
        alpha = 0.5
        beta = 1.0 - alpha
        img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
        img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
        (thresh, img_final_bin) = cv2.threshold(img_final_bin, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        
        contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        (contours, boundingBoxes) = sort_contours(contours,"top-to-bottom")
        
        g = []
        for c in contours:
            x, y, w, h = cv2.boundingRect(c)
            if(w < 800 and w > 500 and h > 220):
                g.append([x, y, w, h])
                
        a = []
        for i in range(len(g)):
            if i == 0:
                a.append(g[i])
            else:
                if np.abs(g[i][0] - g[i-1][0]) < 50:
                    pass
                else:
                    a.append(g[i])
                
        for i in a:
            new_img = img[i[1]:i[1]+i[3], i[0]:i[0]+i[2]]
            dic = {}
            
            extracted_text = pytesseract.image_to_string(new_img, lang='pan', config='--oem 3 --psm 6')
            extracted_text = extracted_text.split()
            
            #Getting name from the list
            start_index_for_name = None
            end_index_for_name = None
            for i, item in enumerate(extracted_text):
                if item.startswith('ਨਾਮ'):
                    start_index_for_name = i
                elif (item.startswith('ਪਿਤਾ') or item.startswith('ਪਤੀ')) and end_index_for_name is None:
                    end_index_for_name = i
                    break
           
            if start_index_for_name is not None and end_index_for_name is not None:
                name = ' '.join(extracted_text[start_index_for_name + 1:end_index_for_name])
                dic["ਨਾਮ"] = str(name)
            else:
                dic["ਨਾਮ"] = ""
                
            #Getting father/husband/mother name from the list
            start_index_for_relation = None
            end_index_for_relation = None
            relation = None
            for i, item in enumerate(extracted_text):
                if item.startswith('ਪਿਤਾ') and start_index_for_relation is None: # or item.startswith('ਪਤੀ')
                    relation = "Father"
                    start_index_for_relation = i
                    break
                elif item.startswith('ਪਤੀ') and start_index_for_relation is None:
                    relation = "Husband"
                    start_index_for_relation = i
                    break
                    
            for i, item in enumerate(extracted_text):
                if (item.startswith('ਫੋਟੋ') or item.startswith('ਮਕਾਨ')) and end_index_for_relation is None:
                    end_index_for_relation = i
                    break
    
            if start_index_for_relation is not None and start_index_for_relation is not None:
                relation_name =  ' '.join(extracted_text[start_index_for_relation + 1:end_index_for_relation])
                patterns = r"ਦਾ ਨਾਮ:|ਤੋਟੇ||ਫੋਟੇ"
                relation_name = re.sub(patterns, "", relation_name)
                
            if relation == "Father":
                dic["ਪਿਤਾ ਦਾ ਨਾਮ"] = str(relation_name)
            else:
                dic["ਪਿਤਾ ਦਾ ਨਾਮ"] = ""
                
            if relation == "Husband":
                dic["ਪਤੀ ਦਾ ਨਾਮ"] = str(relation_name)
            else:
                dic["ਪਤੀ ਦਾ ਨਾਮ"] = ""
            
            print(dic)
            final_list.append(dic)
            
    sd = pd.DataFrame(final_list)
    sd[["ਨਾਮ", "ਪਿਤਾ ਦਾ ਨਾਮ", "ਪਤੀ ਦਾ ਨਾਮ"]].to_excel(str(pdf_file) + ".xlsx")
    
    print(f"{pdf_file} completed")
    
print("===")
print("end")

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'ਨਾਮ': 'ਸਵਿੰਦਰ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਚਰਨ ਸਿੰਘ ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਬਚਨੀ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਫਤਿਹ ਸਿੰਘ ਵੇਦ'}
{'ਨਾਮ': 'ਕੁਲਵੇਤ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਜਵੰਧ ਸਿੰਘ ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਸਵਿੰਦਰ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਕੁਲਵੰਤ ਸਿੰਘ'}
{'ਨਾਮ': 'ਹਰਜਿੰਦਰ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਸਵਿੰਦਰ ਸਿੰਘ'}
{'ਨਾਮ': 'ਵਿਜੇ ਕੁਮਾਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਖਜਾਨ ਚੰਦ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਤਰਸੇਮ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਬੰਤਾ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਬਲਵਿੰਦਰ ਜੀਤ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਥੋੜੂ ਰਾਮ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਕੁਲਦੀਪ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਅਮਰੀਕ ਸਿੰਘ'}
{'ਨਾਮ': 'ਰਾਜ ਰਾਈ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਪ੍ਰਕਾਸ਼ ਚੰਦ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਰਜਵੰਤ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਬਲਵਿੰਦਰ ਜੀਤ'}
{'ਨਾਮ': 'ਵਿਕਰਮਜੀਤ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਸੁਰਿੰਦਰ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਪਵਨਜੀਤ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਗੁਰਮੀਤ ਸਿੰਘ'}
{'ਨਾਮ': 'ਮਨਦੀਪ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਸਵਿੰਦਰ ਸਿੰਘ ਫੋਫੋ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਰਤਿੰਦਰ ਕ

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'ਨਾਮ': 'ਦਵਿੰਦਰ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਵਿਜੇ ਕੁਮਾਰ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਨੇਹਾ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਜਸਪਾਲ ਸਿੰਘ ਫੋਫੋ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': '', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਸੁਖਦੇਵ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਗੁਰਿੰਦਰ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਤਰਸੇਮ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਮੁਲਖ ਰਾਜ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਪਾਰਸ ਰਾਮ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਹਰਭਜਨ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਮਿਲਖਾ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਭਜਨ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਹਰਭਜਨ ਸਿੰਘ'}
{'ਨਾਮ': 'ਮੁਖਤਾਰ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਦਰਸ਼ਨ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਤਰਸੇਮ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਸਵਰਛ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਨਰਿੰਦਰ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਹਰਦਿਆਲ ਸਿੰਘ'}
{'ਨਾਮ': 'ਕੁਲਵੰਤ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਤਾਰਾ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਕਵਲਜੀਤ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਕੁਲਵੰਤ ਸਿੰਘ'}
{'ਨਾਮ': 'ਅਵਤਾਰ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਦਿਆਲ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਨਿਰਮਲਾ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਰਮੇਸ਼ ਕੁਮਾਰ'}
{'ਨਾਮ': 'ਰਮੇਸ਼ ਕੁਮਾਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ'

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

...

{'ਨਾਮ': 'ਰੂਪ ਰਾਨੀ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਜਸਪਾਲ'}
{'ਨਾਮ': 'ਅਸ਼ਵਨੀ ਕੁਮਾਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਜਸਪਾਲ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਮਹਿੰਦਰ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਦਲੀਪ ਚੰਦ'}
{'ਨਾਮ': 'ਬਲਬੀਰ ਚੰਦ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਦਲੀਪ ਚੰਦ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਪਰਮਜੀਤ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਵਿਜੈ ਕੁਮਾਰ'}
{'ਨਾਮ': 'ਵਿਜੈ ਕੁਮਾਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਦਲੀਪ ਚੰਦ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਵੀਨਾ ਕੁਮਾਰੀ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਬਲਬੀਰ ਚੰਦ'}
{'ਨਾਮ': 'ਸਤਪਾਲ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਤੇਜਾ ਰਾਮ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਰਮਨ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਸਤਪਾਲ'}
{'ਨਾਮ': 'ਸਾਹਿਬ ਜੀਤ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਜਸਬੀਰ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਨਿਰਲੇਪ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਸਾਹਿਬ ਜੀਤ ਸਿੰਘ'}
{'ਨਾਮ': 'ਜੈਸਮੀਨ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਦਵਿੰਦਰ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਹਰਨੂਰ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਦਵਿੰਦਰ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਦੀਪਕ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਅਜੀਤ ਕੁਮਾਰ ਸ਼ਰਮਾ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਸੰਦੀਪ ਕੁਮਾਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਪਰਕਾਸ਼ ਚੰਦ

interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'ਨਾਮ': 'ਅਮਰਿੰਦਰ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਉਪਕਾਰ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਕੁਲਵਿੰਦਰ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਅਮਰਿੰਦਰ ਸਿੰਘ ਫੋਫੋ'}
{'ਨਾਮ': 'ਕਿਰਨਦੀਪ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਭੁਪਿੰਦਰ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਕਮਲੇਸ਼ ਰਾਨੀ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਕਸ਼ਮੀਰ ਸਿੰਘ'}
{'ਨਾਮ': 'ਜਤਿੰਦਰ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਕਸ਼ਮੀਰ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਹਰੀਸ਼ ਕਮਾਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਗੋਪਾਲ ਚਾਂਦ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਜੋਏ ਦਾਸ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਤਰਸੇਮ ਮਸੀਹ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਕਸ਼ਮੀਰ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਗੋਪਾਲ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਲਖਬੀਰ ਸਿੰਘ ਬਾਜਵਾ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਪ੍ਰੀਤਮ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਰਜਵੰਤ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਲਖਬੀਰ ਸਿੰਘ'}
{'ਨਾਮ': 'ਗੀਤਾ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਸੁਚਾ ਦਾਸ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਜਸਰੀਤ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਦੁਸ਼ਿਅੰਤ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਜਸਪਾਲ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਗੁਰਭੇਜ ਸਿੰਘ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਕੁਲਵੇਤ ਕੋਰ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਜੋਗਿੰਦਰ ਸਿੰਘ ਫੋਫੋ'}


interactive(children=(IntSlider(value=50, description='x', max=255, min=1), Output()), _dom_classes=('widget-i…

{'ਨਾਮ': 'ਸੁਨਿਤਾ', 'ਪਿਤਾ ਦਾ ਨਾਮ': '', 'ਪਤੀ ਦਾ ਨਾਮ': ' ਰਜਿੰਦਰ ਸਿੰਘ'}
{'ਨਾਮ': 'ਜਸਪਾਲ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਬਿਸ਼ਨ ਲਾਲ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
{'ਨਾਮ': 'ਕਰਨਬੀਰ ਸਿੰਘ', 'ਪਿਤਾ ਦਾ ਨਾਮ': ' ਜੋਗਿੰਦਰ ਪਾਲ', 'ਪਤੀ ਦਾ ਨਾਮ': ''}
1001 completed
===
end
